## Fringe Speech, understanding the language of 4chan during the 2020 election

In [1]:
%load_ext sql
%sql mysql://jeilers@localhost/?charset=utf8mb4

In [2]:
import operator
from termcolor import cprint
from tqdm.auto import tqdm

def write_msgs_from_feat(feat_table, msg_f):
    message_ids = %sql select distinct(group_id) from {feat_table}    
    with open(msgs_f,'w') as msgs_fh:
        for message_id in tqdm(message_ids,desc='msgs written'):
            message_id = message_id[0]
            feats = %sql select feat from {feat_table} where group_id = {message_id}
            feats = map(operator.itemgetter(0),feats)
            feats = ' '.join(feats)
            msgs_fh.write(' '.join([str(message_id),'en',feats]) + '\n')

In [3]:
database = 'jeilers_project'
msgs_table = '4chan2'

feat_1gram_table = 'feat$1gram$4chan2$message_id$16to16'
feat_1gram_pruned_table = 'feat_pruned$1gram$4chan2$message_id$16to16'
feat_1gram_0_0001_table = 'feat_pruned$1gram$4chan2$message_id$16to16$0_0001'

msgs_lda_table = '4chan2_lda$topics'

topics__fb_cp_table = 'topics_fb2k'
topics__fb_freq_table = 'topics_fb2k_freq'
feat_fb_table = 'feat$cat_topics_fb2k_w$4chan2$message_id$1gra'

topics_cp_table = 'blog_authorship_cp'
topic_ll_table = 'blog_authorship_freq'
feat_hour_msg_table = 'feat$cat_blog_authorship_cp_w$4chan2$message_id$1gra'

feat_LIWC_table = 'feat$cat_LIWC2015$4chan2$message_id$1gra'

In [4]:
%sql use {database}

 * mysql://jeilers@localhost/?charset=utf8mb4
0 rows affected.


[]

In [41]:
%sql select count(distinct(feat)) from {feat_1gram_0_0001_table}

 * mysql://jeilers@localhost/?charset=utf8mb4
1 rows affected.


count(distinct(feat))
8273


In [22]:
%%sql
alter table {msgs_table} add primary key message_id (message_id);
alter table {msgs_table} add index post_id (post_id);
CREATE INDEX message ON {msgs_table} (message(100));
alter table {msgs_table} add index date_time (date_time);

 * mysql://jeilers@localhost/?charset=utf8mb4
(MySQLdb._exceptions.OperationalError) (1068, 'Multiple primary key defined')
[SQL: alter table 4chan2 add primary key message_id (message_id);]
(Background on this error at: http://sqlalche.me/e/e3q8)


##  Outline of the steps needed
- 2) dlatk extract 1grams
- 3a) 1grams are written into mallet-readable msgs file in Python
- 3b) mallet converts the above file to its own format for training a topic model  
- ----------------------------------------
- 4) mallet runs LDA and produces files --   **THIS IS THE ACTUAL TOPIC MODELING**  
- ----------------------------------------
- 5a) dlatk converts mallet output to an intermediate file
- 5b) dlatk ingests mallet output files into mysql  
- 5c) dlatk writes topic tables to files -- topic-given-word and word-given-topic
- 5d) dlatk creates the two lexicon tables from the files it just produced -- topic-given-word and word-given-topic

## Extract 1 grams

In [ ]:
!dlatkInterface.py \
    --corpdb {database} \
    --corptable {msgs_table} \
    --correl_field message_id \
    --add_ngrams -n 1

The above command produced table `feat$1gram$4chan2$message_id$16to16`.

In [24]:
%%sql
SET @n = 0;
select @n := @n + 1 n, a.* from 
    (select feat,sum(value) as n_occ from {feat_1gram_table} group by feat order by n_occ desc limit 100) as a

 * mysql://jeilers@localhost/?charset=utf8mb4
0 rows affected.
100 rows affected.


n,feat,n_occ
1,.,48134
2,the,32800
3,",",27035
4,!,21534
5,?,20577
6,to,19578
7,and,17974
8,a,17154
9,is,15293
10,/,14611


Prune the table of its 75 most used words.

In [ ]:
%%sql 
drop table if exists {feat_1gram_pruned_table};
create table {feat_1gram_pruned_table} as
(
    select 
        A.* 
    from 
        {feat_1gram_table} as A,
        (select 
             feat,
             sum(value) as n_occ 
         from {feat_1gram_table} 
         group by feat 
         order by n_occ desc 
         LIMIT 999999999 OFFSET 75) as B
    where A.feat = B.feat
)

## Write 1grams into mallet-formatted msgs file


In [ ]:
out_d = '/home/jeilers/mallet_output'
!mkdir -p {out_d}
!rm -rf {out_d}/*
msgs_f = f'{out_d}/msgs.txt'
%sql alter table {feat_1gram_pruned_table} add index group_id (group_id)
write_msgs_from_feat(feat_1gram_pruned_table,msgs_f)

### mallet formats its msgs file 

In [ ]:
mallet_msgs_f = f'{out_d}/msgs.mallet' 

!/shared/mallet/bin/mallet \
    import-file \
    --input {msgs_f} \
    --output {mallet_msgs_f} \
    --remove-stopwords \
    --keep-sequence

## LDA

In [ ]:
# Files mallet writes as its output after training topics. 
# Please refer to mallet docs for details on this command on what these files contain.
model_f = f'{out_d}/mallet.model'
state_f = f'{out_d}/state.gz'
topic_keys_f = f'{out_d}/topic_keys.txt'
doc_topics_f = f'{out_d}/doc_topics.txt'

!/shared/mallet/bin/mallet \
    train-topics \
    --input {mallet_msgs_f} \
    --num-topics 20 \
    --output-model {model_f} \
    --output-state {state_f} \
    --output-topic-keys {topic_keys_f} \
    --output-doc-topics {doc_topics_f}

## getting Mallet output into DLATK tables



In [ ]:
topics_f = f'{out_d}/topics'

!dlatkInterface.py \
    --add_message_id {msgs_f} {state_f} \
    --output_name {topics_f}

### dlatk ingest Mallet output into mysql to an intermediate table

In [ ]:
!dlatkInterface.py \
    --corpdb {database} \
    --corptable {msgs_table} \
    --add_lda_messages {topics_f}

The above command produced table `4chan2$topics`.

### dlatk writes topic tables to files

In [ ]:
%cd {out_d}

!dlatkInterface.py \
    --corpdb {database} \
    --corptable {msgs_table} \
    --lda_msg_tbl '{msgs_lda_table}' \
    --create_dists

The above command produced csv files with prefix `4chan2*`.

In [ ]:
topic_given_word_f = f'{out_d}/4chan2_lda.topics.topicGivenWord.csv'
loglik_f = f'{out_d}/4chan2_lda.topics.freq.threshed50.loglik.csv'

### dlatk imports topic files into its lexicon database

dlatk can use the csv file `*topicGivenWord.csv` to create the lexicon table topic-given-word -- this is the table we use for extraction of dictionaries. 

In [ ]:
!dlatkInterface.py \
    --lex_interface \
    --topic_csv \
    --topicfile {topic_given_word_f} \
    --create blog_authorship_cp \
    --lexicondb {database}

In [ ]:
%sql select * from blog_authorship_cp order by rand() limit 10

dlatk can use the csv file `*loglik.csv` to create the lexicon table word-given-topic -- this is the table we use for visualization. 

In [ ]:
!dlatkInterface.py \
    --lex_interface \
    --topic_csv \
    --topicfile {loglik_f} \
    --create blog_authorship_freq \
    --lexicondb {database}

In [ ]:
%sql select * from blog_authorship_freq order by rand() limit 10

At this point, the two tables:
- `blog_authorship_cp`
- `blog_authorship_freq`

are what we need for analysis. dlatk can use these two tables to extract topic assignments for a corpus. 

## Topic wordclouds

In [ ]:
%mkdir -p {out_d}/topic_wordclouds
%rm -rf {out_d}/topic_wordclouds/*
%cd {out_d}/topic_wordclouds

!dlatkInterface.py \
    --lexicondb {database} \
    --topic_lexicon {topic_ll_table} \
    --make_all_topic_wordclouds \
    --tagcloud_colorscheme blue \
    --output 'output'

### make columns with time

In [ ]:
%%sql 
alter table {msgs_table} drop column date;
alter table {msgs_table} add column date DATE;
update {msgs_table}
set date = date(date_time)

## filter 1grams

In [ ]:
!dlatkInterface.py \
    --corpdb {database} \
    --corptable {msgs_table} \
    --correl_field message_id \
    --feat_table '{feat_1gram_pruned_table}' \
    --feat_occ_filter --set_p_occ 0.0001

In [ ]:
%sql select count(distinct(feat)) from feat_pruned$1gram$4chan2$message_id$16to16$0_0001

### correlate 1grams with each hour

In [ ]:
out_d = '~/project'
out_name = '1grams'
!mkdir -p {out_d}
!rm -rf {out_d}/{out_name}*

!dlatkInterface.py \
    --corpdb {database} \
    --corptable {msgs_table} \
    --correl_field message_id \
    --correlate --rmatrix --csv --sort \
    --feat_table '{feat_1gram_0_0001_table}' \
    --outcome_table {msgs_table} \
    --outcomes only_hour \
    --categories_to_binary only_hour \
    --tagcloud --make_wordclouds \
    --lexicondb {database} \
    --topic_lexicon {topic_ll_table} \
    --output_name {out_d}/{out_name}

## extract from lexicon, correlate with time

In [ ]:
output = !dlatkInterface.py \
    --lexicondb {database} \
    --corpdb {database} \
    --corptable {msgs_table} \
    --correl_field message_id \
    --add_lex_table -l {topics_cp_table} --weighted_lexicon  2>&1

In [ ]:
out_d = '~/project'
out_name = '1gram_project_hour'
!mkdir -p {out_d}
!rm -rf {out_d}/{out_name}*

!dlatkInterface.py \
    --corpdb {database} \
    --corptable {msgs_table} \
    --correl_field message_id \
    --correlate --rmatrix --csv --sort \
    --outcome_table {msgs_table} \
    --outcomes only_hour \
    --categories_to_binary only_hour \
    --feat_table '{feat_hour_msg_table}' \
    --topic_tagcloud --make_topic_wordclouds \
    --tagcloud_colorscheme blue \
    --lexicondb {database} \
    --topic_lexicon {topic_ll_table} \
    --output_name {out_d}/{out_name}

## extract and correlate LIWC with time

In [ ]:
output = !dlatkInterface.py \
    --corpdb {database} \
    --corptable {msgs_table} \
    --correl_field message_id \
    --add_lex_table -l LIWC2015  2>&1

In [ ]:
open('output.txt','w').write('\n'.join(output))

In [43]:
out_d = '~/project'
out_name = '1gram_project_hour_LIWC'
!mkdir -p {out_d}
!rm -rf {out_d}/{out_name}*

output = !dlatkInterface.py \
    --corpdb {database} \
    --corptable {msgs_table} \
    --correl_field message_id \
    --correlate --rmatrix --csv --sort \
    --feat_table '{feat_LIWC_table}' \
    --outcome_table {msgs_table} \
    --outcomes only_hour \
    --categories_to_binary only_hour \
    --tagcloud --make_wordclouds \
    --tagcloud_colorscheme blue \
    --lexicondb {database} \
    --topic_lexicon {topic_ll_table} \
    --output_name {out_d}/{out_name} 2>&1

In [44]:
open('output.txt','w').write('\n'.join(output))

210589

## trying out fb lexicon

In [32]:
out_d = '~/project'
out_name = '1gram_project_fb'
!mkdir -p {out_d}
!rm -rf {out_d}/{out_name}*

output = !dlatkInterface.py \
    --corpdb {database} \
    --corptable {msgs_table} \
    --correl_field message_id \
    --correlate --rmatrix --csv --sort \
    --feat_table '{feat_fb_table}' \
    --outcome_table {msgs_table} \
    --outcomes only_hour \
    --categories_to_binary only_hour \
    --topic_tagcloud --make_topic_wordclouds \
    --topic_lexicon topics_fb2k_freq \
    --tagcloud_colorscheme blue \
    --output_name {out_d}/{out_name} 2>&1

## find words driving a dictionary

In [38]:
out_d = '~/project'
out_name = '1gram_project_FUNCTION'
!mkdir -p {out_d}
!rm -rf {out_d}/{out_name}*

ouotput = !dlatkInterface.py \
    --corpdb {database} \
    --corptable {msgs_table} \
    --correl_field message_id \
    --correlate --csv --rmatrix \
    --feat_table '{feat_1gram_0_0001_table}' \
    --outcome_table {msgs_table} \
    --outcomes only_hour \
    --categories_to_binary only_hour \
    --tagcloud --make_wordclouds \
    --whitelist --lex_table LIWC2015 --categories 'FUNCTION' \
    --output_name {out_d}/{out_name} 2>&1

In [39]:
open('output.txt','w').write('\n'.join(output))

5294035